In [ ]:
import os
import csv
import sqlite3
import itertools
import time
import math
import statistics
import collections

import igraph
import scipy.special

# import matplotlib.pyplot as plt
# # %matplotlib widget
# %matplotlib inline

In [ ]:
g = igraph.Graph.Read_Pickle(input('Input to read igraph pickle: ').strip())
print(f'Nodes: {len(g.vs)}')
print(f'Edges: {len(g.es)}')

In [ ]:
## Check the bitcoind running
import os

datadir = os.path.abspath(os.path.expanduser('~/.bitcoin'))
pid_path = os.path.join(datadir, 'bitcoind.pid')
if os.path.exists(pid_path):
    with open(pid_path, 'r') as f:
        print(f'Bitcoind PID: {f.read()}')
else:
    raise SystemExit('Bitcoind is not running!')

In [ ]:
# For secrets and Some library
from secret import rpc_user, rpc_password
from address_convertor import pubkey_to_address, get_pubkey
from time_manager import get_time

In [ ]:
from bitcoinrpc.authproxy import AuthServiceProxy, JSONRPCException
rpc_ip = '127.0.0.1'
rpc_port = '8332'
timeout = 60*5

def get_rpc():
    return AuthServiceProxy(f'http://{rpc_user}:{rpc_password}@{rpc_ip}:{rpc_port}', timeout=timeout)

rpc_connection = get_rpc()
best_block_hash = rpc_connection.getbestblockhash()
best_block = rpc_connection.getblock(best_block_hash)
print(f'Best Block Heights: {best_block["height"]}, Time: {get_time(best_block["time"]).isoformat()}')

In [ ]:
DEBUG = True
TX = dict()

# Load database
path = 'file:index.db?mode=ro'
conn = sqlite3.connect(path, uri=True)
cur = conn.cursor()


def get_txin_counts(indices):
    counts = list()
    for n in g.vs.select(indices):
        name = n['name']
        cur.execute('''SELECT COUNT(*) FROM TxIn WHERE addr = ?''', (name,))
        tmp_result = cur.fetchone()
        if tmp_result is not None:
            counts.append(tmp_result[0])
        else:
            counts.append(0)
    return counts


def get_txout_counts(indices):
    counts = list()
    for n in g.vs.select(indices):
        name = n['name']
        cur.execute('''SELECT COUNT(*) FROM TxOut WHERE addr = ?''', (name,))
        tmp_result = cur.fetchone()
        if tmp_result is not None:
            counts.append(tmp_result[0])
        else:
            counts.append(0)
    return counts


def get_in_values(g):
    values = list()
    cnt = 0
    for v in g.vs:
        name = v['name']
        cur.execute('''SELECT TxID.txhash, TxOut.n FROM TxOut 
                           JOIN TxID ON TxOut.tx = TxID.id 
                       WHERE TxOut.addr = ?''', (name,))
        value = 0
        for txhash, n in cur:
            if txhash in TX.keys():
                tx = TX[txhash]
            else:
                tx = rpc_connection.getrawtransaction(txhash, 1)
                TX[txhash] = tx
            value += tx['vout'][n]['value']
        values.append(value)
        if DEBUG:
            cnt += 1
            print(f'[{cnt/len(g.vs):.2f}] {name} {value}', end='\r')
    return values


def get_out_values(g):
    values = list()
    for v in g.vs:
        name = v['name']
        cur.execute('''SELECT TxID.txhash, TxIn.n FROM TxIn 
                           JOIN TxID ON TxIn.tx = TxID.id 
                       WHERE TxIn.addr = ?''', (name,))
        value = 0
        for txhash, n in cur:
            if txhash in TX.keys():
                tx = TX[txhash]
            else:
                tx = rpc_connection.getrawtransaction(txhash, 1)
                TX[txhash] = tx
            if tx['vin'][n]['txid'] in TX.keys():
                ptx = TX[tx['vin'][n]['txid']]
            else:
                ptx = rpc_connection.getrawtransaction(tx['vin'][n]['txid'], 1)
                TX[tx['vin'][n]['txid']] = ptx
            value += ptx['vout'][tx['vin'][n]['vout']]['value']
        values.append(value)
        if DEBUG:
            cnt += 1
            print(f'[{cnt/len(g.vs):.2f}] {name} {value}', end='\r')
    return values


def get_tx_count(g):
    values = list()
    for v in g.vs:
        name = v['name']
        value = 0
        cur.execute('''SELECT COUNT(*) FROM TxIn
                       WHERE TxIn.addr = ?''', (name,))
        value += cur.fetchone()[0]
        cur.execute('''SELECT COUNT(*) FROM TxOut
                       WHERE TxOut.addr = ?''', (name,))
        value += cur.fetchone()[0]
        values.append(value)
        if DEBUG:
            cnt += 1
            print(f'[{cnt/len(g.vs):.2f}] {name} {value}', end='\r')
    return values

In [ ]:
stime = time.time()
rpc_connection = get_rpc()
in_values = get_in_values(g)
etime = time.time()
print(f'In values calculation during {etime-stime}')

In [ ]:
stime = time.time()
rpc_connection = get_rpc()
out_values = get_out_values(g)
etime = time.time()
print(f'Out values calculation during {etime-stime}')

In [ ]:
stime = time.time()
rpc_connection = get_rpc()
tx_count = get_tx_count(g)
etime = time.time()
print(f'Tx Count calculation during {etime-stime}')

In [ ]:
stime = time.time()
indegree = g.indegree()
outdegree = g.outdegree()
etime = time.time()
print(f'Degree during {etime-stime}')

In [ ]:
stime = time.time()
pagerank = g.pagerank()
etime = time.time()
print(f'Pagerank during {etime-stime}')

In [ ]:
import pickle
with open('d.pickle', 'wb') as f:
    pickle.dump(f, (in_values, out_values, indegree, outdegree, pagerank, tx_count))

In [ ]:
# stime = time.time()
# partition = g.community_leiden(objective_function='modularity')
# etime = time.time()
# print(f'{len(partition)} during {etime-stime}')

In [ ]:
# l = [len(p) for p in partition if len(p) >= len(g.vs)*0.01]

In [ ]:
# fig1 = plt.figure(figsize=(4, 4))
# fig1.set_facecolor('white')
# ax1 = fig1.add_subplot()
# ax1.hist(l)
# ax1.set_title('Histogram of cluster size', fontsize='large')
# ax1.set_xlabel('Cluster size', fontsize='x-large')
# ax1.set_ylabel('Cluster count', fontsize='x-large')
# fig1.savefig('histogram.png')

In [ ]:
# with open('meta.csv', 'w') as f:
#     writer = csv.DictWriter(f, fieldnames=['ClusterSize', 
#                                            'TxInMean', 'TxInMedian', 'TxInStdev', 'TxInMin', 'TxInMax', 
#                                            'TxOutMean', 'TxOutMedian', 'TxOutStdev', 'TxOutMin', 'TxOutMax'])
#     writer.writeheader()
#     for i in range(0, len(partition)):
#         p = partition[i]
#         if len(p) < len(g.vs)*0.01:
#             continue
#         txin = get_txin_counts(p)
#         txout = get_txout_counts(p)
#         writer.writerow({'ClusterSize': len(p), 
#                          'TxInMean': statistics.mean(txin),
#                          'TxInMedian': statistics.median(txin),
#                          'TxInStdev': statistics.stdev(txin),
#                          'TxInMin': min(txin),
#                          'TxInMax': max(txin),
#                          'TxOutMean': statistics.mean(txout),
#                          'TxOutMedian': statistics.median(txout),
#                          'TxOutStdev': statistics.stdev(txout),
#                          'TxOutMin': min(txout),
#                          'TxOutMax': max(txout)})

In [ ]:
# stime = time.time()
# rank = g.pagerank()
# etime = time.time()
# print(f'Max pagerank: {max(rank)} at {g.vs[rank.index(max(rank))]["name"]} during {etime-stime}')

In [ ]:
# stime = time.time()
# # https://stackoverflow.com/questions/37855553/python-igraph-community-cluster-colors
# g2 = partition.cluster_graph(combine_vertices='random', combine_edges='sum')
# g2.delete_vertices(g2.vs.select(_degree=0))
# layout = g2.layout_drl()
# igraph.plot(g2, 'partition.svg', layout=layout, vertex_size=5, edge_width=0.5)
# etime = time.time()
# print(f'Plotting done during {etime-stime}')

In [ ]:
conn.close()